In [1]:
import TNG_DA
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

# DS+ and Virial Theorem plots

### Import Data

In [2]:
df_path = "/home/habjan.e/TNG/Data/data_DS+_virial_results/DS+_Virial_df.csv"
df = pd.read_csv(df_path)

In [3]:
df

,Cluster Index,Projection x-Direction,Projection y-Direction,Projection z-Direction,Completeness,Completeness Uncertainty,Purity,Purity Uncertainty,Virial Halo Mass
0,3,-0.595276,-0.860206,0.188564,0.708333,0.037684,0.244604,0.015904,8.226813e+14
1,3,-0.716088,0.636496,-0.622205,0.666667,0.066989,0.222222,0.018903,6.912321e+14
2,3,-0.132594,-0.434955,-0.825670,0.541667,0.047568,0.216667,0.015268,1.117757e+15
3,3,0.897466,-0.672714,-0.300039,0.520833,0.052870,0.206612,0.020174,1.250940e+15
4,3,-0.757156,-0.161701,0.684104,0.729167,0.050112,0.241379,0.022981,1.213218e+15
5,3,0.602978,0.440347,0.723074,0.583333,0.085989,0.224000,0.031334,8.687426e+14
6,3,-0.929623,0.933674,0.138351,0.645833,0.074246,0.212329,0.028189,1.105467e+15
7,3,-0.873153,-0.344202,0.474789,0.708333,0.052937,0.236111,0.023552,1.149277e+15
8,3,0.954052,0.515245,-0.149140,0.750000,0.053049,0.238411,0.025682,9.613217e+14
9,3,0.479113,0.624570,0.127069,0.500000,0.086879,0.193548,0.037439,8.128120e+14


In [4]:
test = np.load('/home/habjan.e/TNG/Data/data_DS+_virial_results/subhalo_masses.npy')

In [5]:
test.shape

(10, 211)